In [5]:
import smtplib
from email.mime.text import MIMEText
from email.header import Header
import pandas as pd
import time
%run schedule_reader.ipynb

In [ ]:
'''# --- 邮件配置 ---
# SMTP 服务器配置
smtp_server = 'smtp.qq.com'  # 例如: smtp.qq.com
smtp_port = 465              # SSL 端口
# 发件人账户信息
sender_email = '2200355981@qq.com'      # 你的邮箱地址
sender_password = 'abcdefghijklmn'     # 你的邮箱授权码，而不是登录密码！'''
#配置区域#
REMINDER_CONFIG = {
    "smtp_server": "smtp.qq.com",
    "smtp_port": 465,
    "sender_email": "2200355981@qq.com",
    "sender_password": "hswdypyrmvjxdjia",
    "receiver_email": "10244801409@stu.ecnu.edu.cn",  # 接收提醒的邮箱
    "check_interval": 60                           # 定时检查间隔（秒）
}

In [ ]:
def send_reminder(receiver_email, course_name, class_time_str, location):
    sender_email = REMINDER_CONFIG["sender_email"]
    sender_password = REMINDER_CONFIG["sender_password"]
    smtp_server = REMINDER_CONFIG["smtp_server"]
    smtp_port = REMINDER_CONFIG["smtp_port"]

    message = MIMEText(
        f'【课程名称】{course_name}\n'
        f'【课程时间】{class_time_str}\n'
        f'【课程地点】{location}'
    )
    message['From'] = Header(sender_email)
    message['To'] = Header(receiver_email)
    message['Subject'] = Header(f'【课程提醒】{course_name}{class_time_str}{location}')

    server = None
    try:
        '''server = smtplib.SMTP('smtp.qq.com', 465)
        server.starttls()'''
        server = smtplib.SMTP_SSL(smtp_server, smtp_port)
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, message.as_string())
        print('邮件发送成功')
    except smtplib.SMTPAuthenticationError:
        print(f'邮件发送失败（{course_name}）：邮箱或授权码错误')
    except smtplib.SMTPConnectError:
        print(f'邮件发送失败（{course_name}）：无法连接到服务器，请检查SMTP地址和端口')
    except Exception as e:
        print(f'邮件发送失败: {e}')
    finally:
        if server is not None:
                server.quit()


In [ ]:
def time_checker(check_interval, raw_courses):
    receiver_email = REMINDER_CONFIG['receiver_email']
    while True:
        current_time = pd.Timestamp.now().tz_localize(None)
        current_weekday = current_time.day_name(locale='zh_CN')
        for x in raw_courses:
            if x['weekday'] != current_weekday:
                continue
            class_time = pd.Timestamp(x['class_time_str'])
            remind_start = class_time - pd.Timedelta(minutes=10)
            if (remind_start <= current_time <= class_time and not x['is_reminded']):
                send_reminder(receiver_email, x['course_name'], x['class_time_str'], x['location'])
                x['is_reminded'] = True
        time.sleep(check_interval)

#current_time = pd.Timestamp.now()生成的时间可能是带时区的，表格里读取后生成的没有，会导致报错Cannot compare tz-naive and tz-aware timestamps